In [1]:
import pandas as pd
import numpy as np
import re
from gensim.models import FastText
from sklearn.cluster import DBSCAN
from sklearn.cluster import MeanShift
from sklearn.cluster import estimate_bandwidth
from collections import defaultdict

# 기본 데이터 EDA

In [2]:
company = pd.read_csv('./MBN00005T.csv',encoding = 'cp949')
company

,FSUJABH,BASEDT1,BASEDT2,PERID,ENTNM,EENTNM,GPCODE,CEONM,BONSA_ADDRESS,BONSA_TELNO,...,YNOTE3,YNOTE4,YNOTE5,YNOTE6,YNOTE7,YNOTE8,YNOTE9,YNOTE10,YNOTE11,YNOTE12
0,30052,20180918,NaN,1.101140e+12,신보희망디딤돌제삼차유동화전문(유),"Hope Kodit Third Securitization Specialty Co.,...",NaN,송위규,"서울특별시 영등포구 은행로 14 (여의도동), 한국산업은행",02-787-6605,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,30071,20181207,NaN,1.101140e+12,트레져씨티유동화전문(유),Treasure City Asset Securitization Specialty C...,NaN,유광균,"서울특별시 중구 을지로 66 (을지로2가), 외환은행본점",02-729-0428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30072,20180918,NaN,1.101140e+12,도공제오차유동화전문(유),KHC 5th Securitization Specialty Co.,NaN,문양주,서울특별시 영등포구 국제금융로8길 26 (여의도동),02-2073-8482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30074,20180918,NaN,1.101140e+12,도공제사차유동화전문(유),KHC 4th Securitization Specialty Co.,NaN,최혜경,서울특별시 영등포구 국제금융로8길 26 (여의도동),02-2073-8492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30080,20180918,NaN,1.101140e+12,산은캐피탈제사차유동화전문(유),NaN,NaN,노미실,서울특별시 영등포구 은행로 14 (여의도동),02-768-3341,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42066,8998700435,20181008,NaN,1.101116e+12,포스트아네모네(주),"POSTANEMONE Co., Ltd",NaN,정양호,"서울특별시 영등포구 여의대방로69길 17(여의도동,메리츠종합금융증권 내)",02-2229-6334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42067,8998700963,20181019,NaN,1.101117e+12,뉴하이미아제일차(주),"NEW HI MIA NO.1 CO,.LTD",NaN,박종필,"서울특별시 영등포구 여의나루로 61 (여의도동, 하이투자증권빌딩)",02-2122-9769,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42068,8998701263,20190404,NaN,1.101117e+12,하나머스트제6호기업인수목적(주),Hana Must Sixth Special Purpose Acquisition Co...,NaN,류제일,서울시 영등포구 의사당대로 82,02-3771-3188,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42069,8998800015,20190131,NaN,1.101116e+12,신한에스러셀(주),"Shinhan S-Russell Co.,Ltd.",NaN,심은정,서울특별시 중구 세종대로9길 20 (태평로2가),02-2151-4968,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
company.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42071 entries, 0 to 42070
Data columns (total 36 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   FSUJABH        42071 non-null  int64  
 1   BASEDT1        42071 non-null  int64  
 2   BASEDT2        6445 non-null   float64
 3   PERID          42029 non-null  float64
 4   ENTNM          42071 non-null  object 
 5   EENTNM         38801 non-null  object 
 6   GPCODE         959 non-null    object 
 7   CEONM          42051 non-null  object 
 8   BONSA_ADDRESS  42061 non-null  object 
 9   BONSA_TELNO    42010 non-null  object 
 10  BONSA_FAXNO    27882 non-null  object 
 11  SAUP_ADDRESS   6445 non-null   object 
 12  SAUP_TELNO     6444 non-null   object 
 13  SAUP_FAXNO     6421 non-null   object 
 14  OPEDT          42058 non-null  float64
 15  ORGBK          6442 non-null   object 
 16  SCALE          42071 non-null  object 
 17  LISTYN         42071 non-null  object 
 18  HOMEPA

## 유용한 컬럼 추출

In [4]:
company_data = company[['ENTNM','BONSA_ADDRESS','SCALE','BISTYPE','GOODNM']]

In [5]:
company_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42071 entries, 0 to 42070
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ENTNM          42071 non-null  object
 1   BONSA_ADDRESS  42061 non-null  object
 2   SCALE          42071 non-null  object
 3   BISTYPE        42071 non-null  object
 4   GOODNM         7319 non-null   object
dtypes: object(5)
memory usage: 1.6+ MB


In [6]:
company_data

,ENTNM,BONSA_ADDRESS,SCALE,BISTYPE,GOODNM
0,신보희망디딤돌제삼차유동화전문(유),"서울특별시 영등포구 은행로 14 (여의도동), 한국산업은행",중소기업,(K67999)유동화전문회사,NaN
1,트레져씨티유동화전문(유),"서울특별시 중구 을지로 66 (을지로2가), 외환은행본점",중소기업,(K67999)유동화전문회사,NaN
2,도공제오차유동화전문(유),서울특별시 영등포구 국제금융로8길 26 (여의도동),중소기업,(K67999)유동화전문회사,NaN
3,도공제사차유동화전문(유),서울특별시 영등포구 국제금융로8길 26 (여의도동),중소기업,(K67999)유동화전문회사,NaN
4,산은캐피탈제사차유동화전문(유),서울특별시 영등포구 은행로 14 (여의도동),중소기업,(K67999)유동화전문회사,NaN
...,...,...,...,...,...
42066,포스트아네모네(주),"서울특별시 영등포구 여의대방로69길 17(여의도동,메리츠종합금융증권 내)",중소기업,(K67998)유동화회사,NaN
42067,뉴하이미아제일차(주),"서울특별시 영등포구 여의나루로 61 (여의도동, 하이투자증권빌딩)",중소기업,(K67998)유동화회사,NaN
42068,하나머스트제6호기업인수목적(주),서울시 영등포구 의사당대로 82,중소기업,(K66199)그 외 기타 금융 지원 서비스업,NaN
42069,신한에스러셀(주),서울특별시 중구 세종대로9길 20 (태평로2가),중소기업,(K67998)유동화회사,NaN


In [7]:
fill_value = ['인천광역시 남동구 논현동 66-12',
                               '경기도 양주시 장흥면 권율로 190-45',
                               '',
                               '',
                               '',
                               '서울특별시 중구 을지로5길 19 (수하동)',
                               '경기도 화성시 반정동 448-8',
                               '서울 영등포구 여의나루로 61',
                               '경상남도 양산시 덕명로 57 (덕계동)',
                               '']

In [8]:
fill_value

['인천광역시 남동구 논현동 66-12',
 '경기도 양주시 장흥면 권율로 190-45',
 '',
 '',
 '',
 '서울특별시 중구 을지로5길 19 (수하동)',
 '경기도 화성시 반정동 448-8',
 '서울 영등포구 여의나루로 61',
 '경상남도 양산시 덕명로 57 (덕계동)',
 '']

In [9]:
company_data = company_data.copy()
company_data.loc[company_data['BONSA_ADDRESS'].isna(),'BONSA_ADDRESS'] = fill_value 

In [10]:
company_data['BONSA_ADDRESS'].isna().sum()

0

In [11]:
company_data = company_data.fillna('')

In [12]:
company_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42071 entries, 0 to 42070
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ENTNM          42071 non-null  object
 1   BONSA_ADDRESS  42071 non-null  object
 2   SCALE          42071 non-null  object
 3   BISTYPE        42071 non-null  object
 4   GOODNM         42071 non-null  object
dtypes: object(5)
memory usage: 1.6+ MB


* 만약 주소가 없다면 폐업으로 간주하고 삭제

In [13]:
for num,add in enumerate(company_data['BONSA_ADDRESS']):
    if add == '':
        company_data.drop(index=num,axis = 0,inplace = True)     

## 회사의 업종과 자세한 품종을 합하기

### Test 자세한 품종을 제외해보기

In [14]:
import re

text = "기타 게임 소프트웨어 개발 및 공급업"

pattern = r"[^ ]+ 및 [^ ]+업|[^ ]+업"
match = re.search(pattern, text)

if match:
    result = match.group()
    print(result)
else:
    print("패턴을 찾을 수 없습니다.")


개발 및 공급업


In [15]:
A = "개발  공급업".replace('및','')
A = A.replace(' ','')
A[:-1]

'개발공급'

In [120]:
new_kind = []
for cat, kind in zip(company_data['BISTYPE'], company_data['GOODNM']):
    cat = re.sub(r"\(.+\)",'',cat)
    cat = re.sub(r"[ㆍ·.,]",' ',cat)
    # pattern = r"[^ ]+ 및 [^ ]+업|[^ ]+업"
    pattern = r"[^ ]+업"
    tmp = re.search(pattern, cat)
    if tmp:
#         cat = tmp.group()
        cat = cat[:-1]
    cat = cat.strip()
    cat = cat.replace('및','')
    cat = cat.replace(' ','')
    new_kind.append(cat)
new_kind

['유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '유동화전문회사',
 '의료용품기타의약관련제품제조',
 '회사본부경영컨설팅서비스',
 '자동차부품판매',
 '조미료식품첨가물제조',
 '승용차기타여객용자동차제조',
 '비금융지주회사',
 '서양식음식점',
 '비주거용건물임대',
 '도로건설',
 '영화관운영',
 '호텔',
 '비주거용건물임대',
 '영화관운영',
 '배관냉난방공사',
 '자동차신품부품제조',
 '부동산임대',
 '기타전자부품제조',
 '내항여객운송',
 '비주거용건물임대',
 '기타부동산임대',
 '건축설계관련서비스',
 '문구용품도매',
 '상호저축은행기타저축기관',
 '종이원지판지종이상자도매',
 '승용차기타여객용자동차제조',
 '여행사',
 '산업용가스제조',
 '일반서적출판',
 '일반국외여행사',
 '상품종합도매',
 '도로건설',
 '자동차신품부품제조',
 '기타기계장비도매',
 '비주거용건물임대',
 '의료기기도매',
 '시멘트제조',
 '기타식료품제조',
 '기초화학물질제조',
 '철근철근콘크리트공사',
 '안경사진장비광학용품도매',
 '상호저축은행기타저축기관',
 '비주거용건물임대',
 '여행사',
 '상품종합도매',
 '예술품골동품소매',
 '예술품골동품소매',
 '생명보험',
 '일반용도료관련제품제조',
 '종이원지판지종이상자도매',
 '악기도매',
 '비주거용건물임대',
 '지반조성건설',
 '골프장운영',
 '종이원지판지종이상자도매',
 '셔츠블라우스제조',
 '비주거용건물임대',
 '부동산자문중개',
 '석탄화학계화합물기타기초유기화학물질제조',
 '외항화물운송',
 '외항화물운송',
 

In [121]:
company_data['company_kind'] = new_kind
company_data_revision = company_data.drop(columns = ['BISTYPE','GOODNM'])
company_data_revision

,ENTNM,BONSA_ADDRESS,SCALE,company_kind
0,신보희망디딤돌제삼차유동화전문(유),"서울특별시 영등포구 은행로 14 (여의도동), 한국산업은행",중소기업,유동화전문회사
1,트레져씨티유동화전문(유),"서울특별시 중구 을지로 66 (을지로2가), 외환은행본점",중소기업,유동화전문회사
2,도공제오차유동화전문(유),서울특별시 영등포구 국제금융로8길 26 (여의도동),중소기업,유동화전문회사
3,도공제사차유동화전문(유),서울특별시 영등포구 국제금융로8길 26 (여의도동),중소기업,유동화전문회사
4,산은캐피탈제사차유동화전문(유),서울특별시 영등포구 은행로 14 (여의도동),중소기업,유동화전문회사
...,...,...,...,...
42066,포스트아네모네(주),"서울특별시 영등포구 여의대방로69길 17(여의도동,메리츠종합금융증권 내)",중소기업,유동화회사
42067,뉴하이미아제일차(주),"서울특별시 영등포구 여의나루로 61 (여의도동, 하이투자증권빌딩)",중소기업,유동화회사
42068,하나머스트제6호기업인수목적(주),서울시 영등포구 의사당대로 82,중소기업,그외기타금융지원서비스
42069,신한에스러셀(주),서울특별시 중구 세종대로9길 20 (태평로2가),중소기업,유동화회사


# company_kind DBSCAN 수행

In [122]:
company_kind = company_data_revision['company_kind']
company_kind

0            유동화전문회사
1            유동화전문회사
2            유동화전문회사
3            유동화전문회사
4            유동화전문회사
            ...     
42066          유동화회사
42067          유동화회사
42068    그외기타금융지원서비스
42069          유동화회사
42070          경영컨설팅
Name: company_kind, Length: 42067, dtype: object

In [123]:
company_idx_dict = {}
group_num = 0
for kind in company_kind:
    if kind not in company_idx_dict:
        company_idx_dict[kind] = group_num
        group_num+=1
company_idx_dict

{'유동화전문회사': 0,
 '의료용품기타의약관련제품제조': 1,
 '회사본부경영컨설팅서비스': 2,
 '자동차부품판매': 3,
 '조미료식품첨가물제조': 4,
 '승용차기타여객용자동차제조': 5,
 '비금융지주회사': 6,
 '서양식음식점': 7,
 '비주거용건물임대': 8,
 '도로건설': 9,
 '영화관운영': 10,
 '호텔': 11,
 '배관냉난방공사': 12,
 '자동차신품부품제조': 13,
 '부동산임대': 14,
 '기타전자부품제조': 15,
 '내항여객운송': 16,
 '기타부동산임대': 17,
 '건축설계관련서비스': 18,
 '문구용품도매': 19,
 '상호저축은행기타저축기관': 20,
 '종이원지판지종이상자도매': 21,
 '여행사': 22,
 '산업용가스제조': 23,
 '일반서적출판': 24,
 '일반국외여행사': 25,
 '상품종합도매': 26,
 '기타기계장비도매': 27,
 '의료기기도매': 28,
 '시멘트제조': 29,
 '기타식료품제조': 30,
 '기초화학물질제조': 31,
 '철근철근콘크리트공사': 32,
 '안경사진장비광학용품도매': 33,
 '예술품골동품소매': 34,
 '생명보험': 35,
 '일반용도료관련제품제조': 36,
 '악기도매': 37,
 '지반조성건설': 38,
 '골프장운영': 39,
 '셔츠블라우스제조': 40,
 '부동산자문중개': 41,
 '석탄화학계화합물기타기초유기화학물질제조': 42,
 '외항화물운송': 43,
 '1차금속제품도매': 44,
 '응용소프트웨어개발공급': 45,
 '일반교과학원': 46,
 '편조의복제조': 47,
 '휴양콘도운영': 48,
 '기타상품중개': 49,
 '기타토목시설물건설': 50,
 '컴퓨터시스템통합자문구축서비스': 51,
 '공기조화장치제조': 52,
 '비주거용건물개발공급': 53,
 '금속절삭기계제조': 54,
 '광고대행': 55,
 '축전지제조': 56,
 '금속탱크저장용기제조': 57,
 '방문판매': 58,
 '부동산관련서비스': 59,
 '의학약학

In [124]:
len(company_idx_dict)

1584

* 242개의 종류의 회사

## 사전훈련된 한국어 FastText를 이용해 벡터화

In [125]:
fasttext = FastText.load_fasttext_format('./ft.morpheme.char.w+namu+nsmc.bin')

/tmp/ipykernel_1746019/2451447361.py:1: DeprecationWarning: Call to deprecated `load_fasttext_format` (use load_facebook_vectors (to use pretrained embeddings) or load_facebook_model (to continue training with the loaded full model, more RAM) instead).
  fasttext = FastText.load_fasttext_format('./ft.morpheme.char.w+namu+nsmc.bin')


In [126]:
vec_list = []
kind_list = [] # 8498개의 종류의 회사에 대해서 클러스터링 수행
for kind in company_kind:
    if company_idx_dict[kind] not in kind_list: 
        kind_list.append(company_idx_dict[kind])
        tmp = fasttext.wv[kind]
        vec_list.append(tmp)
vec_list

[array([ 4.15717922e-02,  2.88253486e-01, -2.74302363e-01,  3.93436998e-02,
         2.78186351e-01, -8.21866468e-03,  1.99300781e-01,  2.00237215e-01,
         3.83686088e-02,  3.25016044e-02, -6.06816448e-02,  2.71988511e-01,
        -1.44420996e-01, -3.03736001e-01, -7.68807903e-02, -1.05006054e-01,
         3.98409329e-02,  1.05923504e-01,  3.22981954e-01, -1.76065192e-01,
        -1.06639557e-01, -1.24996729e-01,  3.91827188e-02,  9.49431583e-03,
        -3.57166260e-01,  1.14006639e-01, -3.64750415e-01,  4.37737331e-02,
        -6.60672262e-02, -8.39444101e-02,  2.50203520e-01, -2.54216462e-01,
         1.18710823e-01, -9.69580337e-02, -1.77846029e-01,  1.10286757e-01,
        -1.08421281e-01, -1.88536420e-01,  8.99222568e-02, -2.62738410e-02,
         3.28789324e-01, -2.22384989e-01,  1.38689167e-04,  3.85706395e-01,
        -6.81612827e-03, -2.15902656e-01,  2.14807883e-01,  1.01678655e-01,
        -2.03797385e-01,  1.88002735e-01,  3.88353206e-02, -2.46229410e-01,
        -9.2

In [127]:
len(kind_list)

1584

In [128]:
kind_list[:5] # vec_list의 각 종류를 저장

[0, 1, 2, 3, 4]

In [129]:
vec_array = np.array(vec_list)
vec_array[:2]

array([[ 4.15717922e-02,  2.88253486e-01, -2.74302363e-01,
         3.93436998e-02,  2.78186351e-01, -8.21866468e-03,
         1.99300781e-01,  2.00237215e-01,  3.83686088e-02,
         3.25016044e-02, -6.06816448e-02,  2.71988511e-01,
        -1.44420996e-01, -3.03736001e-01, -7.68807903e-02,
        -1.05006054e-01,  3.98409329e-02,  1.05923504e-01,
         3.22981954e-01, -1.76065192e-01, -1.06639557e-01,
        -1.24996729e-01,  3.91827188e-02,  9.49431583e-03,
        -3.57166260e-01,  1.14006639e-01, -3.64750415e-01,
         4.37737331e-02, -6.60672262e-02, -8.39444101e-02,
         2.50203520e-01, -2.54216462e-01,  1.18710823e-01,
        -9.69580337e-02, -1.77846029e-01,  1.10286757e-01,
        -1.08421281e-01, -1.88536420e-01,  8.99222568e-02,
        -2.62738410e-02,  3.28789324e-01, -2.22384989e-01,
         1.38689167e-04,  3.85706395e-01, -6.81612827e-03,
        -2.15902656e-01,  2.14807883e-01,  1.01678655e-01,
        -2.03797385e-01,  1.88002735e-01,  3.88353206e-0

In [220]:
dbscan = DBSCAN(eps = 0.135,min_samples = 2, metric = 'cosine')
# dbscan = DBSCAN(eps = 0.14,min_samples = 2, metric = 'cosine')
predict_array = dbscan.fit_predict(vec_array)

* 몇개의 종류가 나오는지 확인

In [221]:
tmp_df = pd.DataFrame({'cluster':predict_array})
tmp_df

,cluster
0,-1
1,-1
2,0
3,1
4,-1
...,...
1579,-1
1580,111
1581,-1
1582,-1


In [222]:
tmp_df.nunique()

cluster    116
dtype: int64

## 같은 클러스터로 묶인 업종 확인

In [223]:
len(predict_array)

1584

In [224]:
kind_array = np.array(kind_list) # 데이터 형태를 맞추기 위해 array로 변형

In [225]:
group_dict = defaultdict(set)

for real,predict in zip(kind_array,predict_array):
    if predict not in group_dict: # 만약 group_dict에 존재하지 않을경우
        tmp_set = set()
        tmp_set.add(real)
        group_dict[predict] = tmp_set
    else:
        group_dict[predict].add(real)
group_dict

defaultdict(set,
            {-1: {0,
              1,
              4,
              7,
              8,
              10,
              11,
              12,
              14,
              16,
              19,
              21,
              22,
              24,
              25,
              26,
              28,
              29,
              32,
              34,
              35,
              36,
              37,
              39,
              41,
              44,
              46,
              47,
              48,
              49,
              52,
              54,
              55,
              56,
              58,
              59,
              61,
              63,
              64,
              65,
              68,
              69,
              70,
              75,
              76,
              77,
              80,
              81,
              82,
              83,
              84,
              85,
              88,
              90,
            

* 한개로 표시된 것들은 실제로 같은 그룹인 경우

## 어떤 그룹이 묶였는지 확인

In [226]:
company_idx_dict

{'유동화전문회사': 0,
 '의료용품기타의약관련제품제조': 1,
 '회사본부경영컨설팅서비스': 2,
 '자동차부품판매': 3,
 '조미료식품첨가물제조': 4,
 '승용차기타여객용자동차제조': 5,
 '비금융지주회사': 6,
 '서양식음식점': 7,
 '비주거용건물임대': 8,
 '도로건설': 9,
 '영화관운영': 10,
 '호텔': 11,
 '배관냉난방공사': 12,
 '자동차신품부품제조': 13,
 '부동산임대': 14,
 '기타전자부품제조': 15,
 '내항여객운송': 16,
 '기타부동산임대': 17,
 '건축설계관련서비스': 18,
 '문구용품도매': 19,
 '상호저축은행기타저축기관': 20,
 '종이원지판지종이상자도매': 21,
 '여행사': 22,
 '산업용가스제조': 23,
 '일반서적출판': 24,
 '일반국외여행사': 25,
 '상품종합도매': 26,
 '기타기계장비도매': 27,
 '의료기기도매': 28,
 '시멘트제조': 29,
 '기타식료품제조': 30,
 '기초화학물질제조': 31,
 '철근철근콘크리트공사': 32,
 '안경사진장비광학용품도매': 33,
 '예술품골동품소매': 34,
 '생명보험': 35,
 '일반용도료관련제품제조': 36,
 '악기도매': 37,
 '지반조성건설': 38,
 '골프장운영': 39,
 '셔츠블라우스제조': 40,
 '부동산자문중개': 41,
 '석탄화학계화합물기타기초유기화학물질제조': 42,
 '외항화물운송': 43,
 '1차금속제품도매': 44,
 '응용소프트웨어개발공급': 45,
 '일반교과학원': 46,
 '편조의복제조': 47,
 '휴양콘도운영': 48,
 '기타상품중개': 49,
 '기타토목시설물건설': 50,
 '컴퓨터시스템통합자문구축서비스': 51,
 '공기조화장치제조': 52,
 '비주거용건물개발공급': 53,
 '금속절삭기계제조': 54,
 '광고대행': 55,
 '축전지제조': 56,
 '금속탱크저장용기제조': 57,
 '방문판매': 58,
 '부동산관련서비스': 59,
 '의학약학

In [227]:
# index와 company를 뒤집음
idx_company_dict = {value: key for key,value in company_idx_dict.items()}
idx_company_dict

{0: '유동화전문회사',
 1: '의료용품기타의약관련제품제조',
 2: '회사본부경영컨설팅서비스',
 3: '자동차부품판매',
 4: '조미료식품첨가물제조',
 5: '승용차기타여객용자동차제조',
 6: '비금융지주회사',
 7: '서양식음식점',
 8: '비주거용건물임대',
 9: '도로건설',
 10: '영화관운영',
 11: '호텔',
 12: '배관냉난방공사',
 13: '자동차신품부품제조',
 14: '부동산임대',
 15: '기타전자부품제조',
 16: '내항여객운송',
 17: '기타부동산임대',
 18: '건축설계관련서비스',
 19: '문구용품도매',
 20: '상호저축은행기타저축기관',
 21: '종이원지판지종이상자도매',
 22: '여행사',
 23: '산업용가스제조',
 24: '일반서적출판',
 25: '일반국외여행사',
 26: '상품종합도매',
 27: '기타기계장비도매',
 28: '의료기기도매',
 29: '시멘트제조',
 30: '기타식료품제조',
 31: '기초화학물질제조',
 32: '철근철근콘크리트공사',
 33: '안경사진장비광학용품도매',
 34: '예술품골동품소매',
 35: '생명보험',
 36: '일반용도료관련제품제조',
 37: '악기도매',
 38: '지반조성건설',
 39: '골프장운영',
 40: '셔츠블라우스제조',
 41: '부동산자문중개',
 42: '석탄화학계화합물기타기초유기화학물질제조',
 43: '외항화물운송',
 44: '1차금속제품도매',
 45: '응용소프트웨어개발공급',
 46: '일반교과학원',
 47: '편조의복제조',
 48: '휴양콘도운영',
 49: '기타상품중개',
 50: '기타토목시설물건설',
 51: '컴퓨터시스템통합자문구축서비스',
 52: '공기조화장치제조',
 53: '비주거용건물개발공급',
 54: '금속절삭기계제조',
 55: '광고대행',
 56: '축전지제조',
 57: '금속탱크저장용기제조',
 58: '방문판매',
 59: '부동산관련서비스',
 60: '

In [228]:
group_dict_kind_ver = {}
for key,value in group_dict.items():
    temp = set()
    for cluster in value:
        temp.add(idx_company_dict[cluster])
    group_dict[key] = temp
group_dict

defaultdict(set,
            {-1: {'',
              '보증보험',
              '육우사육',
              '금속열처리도금기타금속가공',
              '통신기기소매',
              '투자자문',
              '얼음제조',
              '인력공급',
              '커피가공',
              '윤활유그리스제조',
              '자동차수리세차',
              '콘크리트타일기와벽돌블록제조',
              '골판지골판지가공제품제조',
              '김치류제조',
              '판유리제조',
              '증류주합성주제조',
              '매트리스침대제조',
              '그외기타숙박',
              '건설장비운영',
              '방음내화공사',
              '여관',
              '예식장',
              '모방적',
              '펄프종이판지제조',
              '그외기타개인가정용품수리',
              '볼베어링롤러베어링제조',
              '기타환경정화복원',
              '의약품의료용품소매',
              '농림업용기계장비도매',
              '의료기기도매',
              '곡물도정',
              '절삭가공유사처리',
              '자수제품자수용재료제조',
              '사회복지서비스',
              '가금류조류사육',
              '고무제품제조',
              '중고자동차판매',
              '셔츠체육복제조',
              '자동차신품판매',
             

# Clustering한 회사 데이터 확인

In [230]:
company_data_revision

,ENTNM,BONSA_ADDRESS,SCALE,company_kind
0,신보희망디딤돌제삼차유동화전문(유),"서울특별시 영등포구 은행로 14 (여의도동), 한국산업은행",중소기업,유동화전문회사
1,트레져씨티유동화전문(유),"서울특별시 중구 을지로 66 (을지로2가), 외환은행본점",중소기업,유동화전문회사
2,도공제오차유동화전문(유),서울특별시 영등포구 국제금융로8길 26 (여의도동),중소기업,유동화전문회사
3,도공제사차유동화전문(유),서울특별시 영등포구 국제금융로8길 26 (여의도동),중소기업,유동화전문회사
4,산은캐피탈제사차유동화전문(유),서울특별시 영등포구 은행로 14 (여의도동),중소기업,유동화전문회사
...,...,...,...,...
42066,포스트아네모네(주),"서울특별시 영등포구 여의대방로69길 17(여의도동,메리츠종합금융증권 내)",중소기업,유동화회사
42067,뉴하이미아제일차(주),"서울특별시 영등포구 여의나루로 61 (여의도동, 하이투자증권빌딩)",중소기업,유동화회사
42068,하나머스트제6호기업인수목적(주),서울시 영등포구 의사당대로 82,중소기업,그외기타금융지원서비스
42069,신한에스러셀(주),서울특별시 중구 세종대로9길 20 (태평로2가),중소기업,유동화회사


In [231]:
company_data_revision['company_kind'] = new_kind

In [235]:
# 각 업종별 그룹 딕셔너리 생성
idx_group_dict = {}
for key,value_set in group_dict.items():
    for value in value_set:
        idx_group_dict[value] = key
idx_group_dict

{'': -1,
 '보증보험': -1,
 '육우사육': -1,
 '금속열처리도금기타금속가공': -1,
 '통신기기소매': -1,
 '투자자문': -1,
 '얼음제조': -1,
 '인력공급': -1,
 '커피가공': -1,
 '윤활유그리스제조': -1,
 '자동차수리세차': -1,
 '콘크리트타일기와벽돌블록제조': -1,
 '골판지골판지가공제품제조': -1,
 '김치류제조': -1,
 '판유리제조': -1,
 '증류주합성주제조': -1,
 '매트리스침대제조': -1,
 '그외기타숙박': -1,
 '건설장비운영': -1,
 '방음내화공사': -1,
 '여관': -1,
 '예식장': -1,
 '모방적': -1,
 '펄프종이판지제조': -1,
 '그외기타개인가정용품수리': -1,
 '볼베어링롤러베어링제조': -1,
 '기타환경정화복원': -1,
 '의약품의료용품소매': -1,
 '농림업용기계장비도매': -1,
 '의료기기도매': -1,
 '곡물도정': -1,
 '절삭가공유사처리': -1,
 '자수제품자수용재료제조': -1,
 '사회복지서비스': -1,
 '가금류조류사육': -1,
 '고무제품제조': -1,
 '중고자동차판매': -1,
 '셔츠체육복제조': -1,
 '자동차신품판매': -1,
 '태양력발전': -1,
 '뉴스제공': -1,
 '그외기타분류안된가정용품소매': -1,
 '기타내륙수상항만내운송': -1,
 '철도여객운송': -1,
 '전시컨벤션행사대행': -1,
 '부동산중개대리': -1,
 '이동전화기제조': -1,
 '방직용섬유실직물도매': -1,
 '시외버스운송': -1,
 '식물원동물원운영': -1,
 '일반음식점': -1,
 '내부통신배선공사': -1,
 '한식육류요리전문점': -1,
 '배합사료제조': -1,
 '질소인산칼리질비료제조': -1,
 '공인회계사': -1,
 '남녀용겉옷셔츠도매': -1,
 '광물처리취급장비제조': -1,
 '일반도자기제조': -1,
 '기타음향기기제조': -1,
 '시멘트제조': -1,
 '놀이터용장비제조': -1,


In [236]:
# 각 회사별 그룹 리스트 생성
company_group_list = []
for company_dt in company_data_revision['company_kind']:
    company_group_list.append(idx_group_dict[company_dt])
company_group_list

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 1,
 -1,
 2,
 3,
 -1,
 -1,
 4,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 2,
 -1,
 -1,
 5,
 6,
 -1,
 7,
 -1,
 2,
 -1,
 8,
 -1,
 -1,
 -1,
 4,
 1,
 9,
 -1,
 -1,
 -1,
 2,
 10,
 -1,
 11,
 7,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 4,
 -1,
 -1,
 12,
 -1,
 -1,
 2,
 13,
 13,
 -1,
 -1,
 9,
 14,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 13,
 -1,
 4,
 -1,
 -1,
 15,
 9,
 -1,
 16,
 -1,
 -1,
 -1,
 -1,
 13,
 17,
 -1,
 -1,
 5,
 -1,
 -1,
 18,
 14,
 -1,
 19,
 14,
 -1,
 -1,
 -1,
 6,
 2,
 -1,
 -1,
 -1,
 13,
 -1,
 -1,
 14,
 -1,
 19,
 -1,
 14,
 20,
 6,
 21,
 -1,
 -1,
 -1,
 2,
 22,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 23,
 -1,
 24,
 -1,
 -1,
 6,
 25,
 -1,
 -1,
 -1,
 -1,
 18,
 -1,
 -1,
 14,
 -1,
 -1,
 26,
 -1,
 -1,
 27,
 16,
 -1,
 18,
 -1,
 14,
 -1,
 -1,
 -1,
 6,
 28,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 19,
 18,
 -1,
 6,
 14,
 6,
 21,
 -1,
 6,
 14,
 -1,
 -1,
 -1,
 6,
 -1,
 -1,
 -1,
 6,
 -1,
 -1,

In [237]:
company_data_revision['group'] = company_group_list
company_data_revision

,ENTNM,BONSA_ADDRESS,SCALE,company_kind,group
0,신보희망디딤돌제삼차유동화전문(유),"서울특별시 영등포구 은행로 14 (여의도동), 한국산업은행",중소기업,유동화전문회사,-1
1,트레져씨티유동화전문(유),"서울특별시 중구 을지로 66 (을지로2가), 외환은행본점",중소기업,유동화전문회사,-1
2,도공제오차유동화전문(유),서울특별시 영등포구 국제금융로8길 26 (여의도동),중소기업,유동화전문회사,-1
3,도공제사차유동화전문(유),서울특별시 영등포구 국제금융로8길 26 (여의도동),중소기업,유동화전문회사,-1
4,산은캐피탈제사차유동화전문(유),서울특별시 영등포구 은행로 14 (여의도동),중소기업,유동화전문회사,-1
...,...,...,...,...,...
42066,포스트아네모네(주),"서울특별시 영등포구 여의대방로69길 17(여의도동,메리츠종합금융증권 내)",중소기업,유동화회사,-1
42067,뉴하이미아제일차(주),"서울특별시 영등포구 여의나루로 61 (여의도동, 하이투자증권빌딩)",중소기업,유동화회사,-1
42068,하나머스트제6호기업인수목적(주),서울시 영등포구 의사당대로 82,중소기업,그외기타금융지원서비스,6
42069,신한에스러셀(주),서울특별시 중구 세종대로9길 20 (태평로2가),중소기업,유동화회사,-1


In [240]:
company_data_revision['group'].value_counts()

-1      27472
 2       2728
 4       2064
 16      1703
 14       762
        ...  
 105        4
 97         4
 108        4
 72         4
 112        3
Name: group, Length: 116, dtype: int64